# Projeto 1 - Ciência dos Dados

Nome: Maria Eduarda Gonçalves Torres

Nome: Pedro Rubens 

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [374]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 


In [375]:
def cleanup(text):    
    """        Função de limpeza muito simples que troca alguns sinais básicos por espaços    """    
    import string   
    tira_mencoes = re.sub("@[A-Za-z0-9_]+","", text)  
    punctuation = '[!-.:?;@\/…《》]' # Note que os sinais [] são delimitadores de um conjunto.    
    pattern = re.compile(punctuation)   
    text_subbed = re.sub(pattern, ' ', tira_mencoes)   
    return text_subbed

In [376]:
filename = 'Spotify.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Spotify.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [377]:
train = pd.read_excel('Spotify.xlsx')
train.head()

,Treinamento,Classificacao
0,rt @clarisdsf: cade “barquinho de papel” no sp...,0
1,rt @sugaminart: as us armys tão falando muito ...,0
2,gatilhoooooo https://t.co/fronb3zngb,1
3,nem tenho o que dizer https://t.co/9dlkkj5urh,0
4,putos dos anúncios da colgate que aparecem no ...,0


In [378]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test

,Teste,Classificacao
0,aqui está uma música para você… the carol de l...,0
1,hoje essa daqui direto da adolescência https:/...,0
2,aqui está uma música para você… dynamite de bt...,1
3,acordei c essa na cabeça https://t.co/r4audermuo,0
4,a propagando da colgate no spotify é péssima m...,0
...,...,...
113,rt @btspurplebrasil: [info📍]\n\nhot 100 top6 n...,0
114,https://t.co/ss8jb1hrlw bom dia brasil,0
115,vibe alta pra trabalhar bem\nhttps://t.co/2kni...,0
116,rt @upvittarr: envelheceu como vinho. \nhttps:...,0


___
## Classificador automático de sentimento


Neste trabalho, pensamos da segunda pergunta:
O quanto os usuarios estão ajudando a promover o aplicativo Spotify?

Vamos classificar como relevante tweets que falam sobre o app e seu serviço. E como irrelevante tweets que só compartilham musicas e não mencionam o nome do aplicativo.

## Limpeza das bases

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [379]:
R = train['Classificacao'] == 1
relevante = train.loc[R,:]


In [380]:
I = train['Classificacao'] == 0
irrelevante = train.loc[I,:]


##### Total de ocorrencias


In [381]:
total = len(train.index)
total

241

In [382]:
total_relevante = len(relevante.index)
total_relevante

75

In [383]:
total_irrelevante = len(irrelevante.index)
total_irrelevante

166

##### probabilidade de ser relevante

In [384]:
prob_relevante= total_relevante/total
prob_relevante

0.3112033195020747

##### probabilidade de ser irrelevante

In [385]:
prob_irrelevante= total_irrelevante/total
prob_irrelevante

0.6887966804979253

##### transformando em texto

In [386]:
texto=""
for e in train.Treinamento:
    texto+= " " + e 


In [387]:
texto_relevante=""
for e in relevante.Treinamento:
    texto_relevante+= " " + e 

In [388]:
texto_irrelevante=""
for e in irrelevante.Treinamento:
    texto_irrelevante+= " " + e 

##### limpeza das bases

##### Split and cleanup

In [389]:

texto = cleanup(texto.lower())
texto_split = pd.Series(texto.split())

texto_relevante=cleanup(texto_relevante.lower())
texto_relevante_split=pd.Series(texto_relevante.lower())

texto_irrelevante=cleanup(texto_irrelevante.lower())
texto_irrelevante_split=pd.Series(texto_irrelevante.lower())


In [390]:
for e in relevante.Treinamento:
    e = cleanup(e.lower())
    x = re.sub("@[A-Za-z0-9]+","",e)
x

'aqui está uma música para você \n p i m p  de 50 cent 🎶\nhttps   t co xzq0gujavv'

In [391]:
for e in irrelevante.Treinamento:
    e = cleanup(e.lower())
    x = re.sub("@[A-Za-z0-9]+","",e)
x

'o cabelo da jiwoo nessa era só tava o bagaco qria ate entrega um shampoo do futuro p ela\n https   t co bpvxfmih37'

In [392]:
serie_relevante = pd.Series(texto_relevante.split())
serie_relevante.head(20)

0     gatilhoooooo
1            https
2                t
3               co
4       fronb3zngb
5              deu
6             tudo
7        ceeertooo
8                e
9            vamos
10              de
11         spotify
12           hummm
13             vou
14              lá
15              no
16         spotify
17             dar
18             uma
19          ouvida
dtype: object

In [393]:
serie_irrelevante = pd.Series(texto_irrelevante.split())
serie_irrelevante.head(20)

0             rt
1           cade
2     “barquinho
3             de
4         papel”
5             no
6        spotify
7           hein
8         espero
9            por
10          isso
11           faz
12             2
13          anos
14            😔😔
15            rt
16            as
17            us
18         armys
19           tão
dtype: object

#### Frequências absolutas das palavras

In [394]:
tabela_relativa = serie_relevante.value_counts()
tabela_relativa 

https         54
t             51
co            51
de            49
spotify       34
              ..
😭😭😭❤           1
ogcylzowre     1
coisas         1
atualmente     1
disponível     1
Length: 596, dtype: int64

In [395]:
tabela_irrelativa = serie_irrelevante.value_counts()
tabela_irrelativa 

t             126
https         126
co            126
de             61
a              57
             ... 
gcwjr3lcsx      1
rhljgkj5vf      1
futuro          1
estar           1
disponível      1
Length: 1025, dtype: int64

#### Frequência relativa das palavras

In [396]:
tabela_relevante_relativa = serie_relevante.value_counts(True)
tabela_relevante_relativa

https         0.039387
t             0.037199
co            0.037199
de            0.035740
spotify       0.024799
                ...   
😭😭😭❤          0.000729
ogcylzowre    0.000729
coisas        0.000729
atualmente    0.000729
disponível    0.000729
Length: 596, dtype: float64

In [397]:
tabela_irrelevante_relativa = serie_irrelevante.value_counts(True)
tabela_irrelevante_relativa

t             0.050663
https         0.050663
co            0.050663
de            0.024528
a             0.022919
                ...   
gcwjr3lcsx    0.000402
rhljgkj5vf    0.000402
futuro        0.000402
estar         0.000402
disponível    0.000402
Length: 1025, dtype: float64

In [398]:
tabela_irrelevante_relativa["https"]

0.05066344993968637

In [399]:
tabela_relevante_relativa["spotify"]

0.024799416484318017

#### Interseção entre os conjuntos

In [400]:
set_relevante = set(tabela_relevante_relativa.index)
set_irrelevante = set(tabela_irrelevante_relativa.index)
inter = set_relevante.intersection(set_irrelevante)


In [401]:
spotify = texto_irrelevante + texto_relevante
serie_spotify = pd.Series(spotify.split())

tabela_spotify_absolut = serie_spotify .value_counts()

ppos = tabela_relativa.sum()/tabela_spotify_absolut.sum()
pneg = tabela_irrelativa.sum()/tabela_spotify_absolut.sum()

#### Classificando frases

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [402]:
base_teste = pd.ExcelFile('Spotify.xlsx')
df1 = pd.read_excel(base_teste,'Teste')
df1.iloc[90]

Teste            rt @reputaylor: o maior ato da indústria music...
Classificacao                                                    0
Name: 90, dtype: object

In [403]:
# Percorrendo linha a linha e calculando probabilidades
i=0
for e in df1.Teste: 
    e = cleanup(e.lower())
    x = re.sub("@[A-Za-z0-9]+","",e)
    tweets = pd.Series(x.split())
    
    relev = 1
    for p in tweets:
        relev *= tabela_relevante_relativa.get(p, 1)

    irrelev = 1   
    for p in tweets:
        irrelev *= tabela_irrelevante_relativa.get(p, 1)
    
  
    
    relevantee = ppos*relev
    irrelevantee = pneg*irrelev
    
    if relev > irrelev:
        classif = 1
    elif irrelev > relev :
        classif = 0
  
        
  
    df1.loc[i, "classif"] = classif
    i+=1
    
print(relev, irrelev)

print (df1)

1.4887568705236534e-23 2.2484029865226378e-21
                                                 Teste  Classificacao  classif
0    aqui está uma música para você… the carol de l...              0      1.0
1    hoje essa daqui direto da adolescência https:/...              0      1.0
2    aqui está uma música para você… dynamite de bt...              1      0.0
3    acordei c essa na cabeça  https://t.co/r4audermuo              0      0.0
4    a propagando da colgate no spotify é péssima m...              0      1.0
..                                                 ...            ...      ...
113  rt @btspurplebrasil: [info📍]\n\nhot 100 top6 n...              0      1.0
114             https://t.co/ss8jb1hrlw bom dia brasil              0      0.0
115  vibe alta pra trabalhar bem\nhttps://t.co/2kni...              0      1.0
116  rt @upvittarr: envelheceu como vinho. \nhttps:...              0      0.0
117  rt @pedrorhuas: “canção da esperança” inaugura...              0      0.0

[118 

In [404]:
comparacao = (pd.crosstab(df1['Classificacao'], df1['classif']))
comparacao

classif,0.0,1.0
Classificacao,,
0,34,39
1,16,29


In [405]:
j = 0
total = 0
while j < 118:
    if df1.Classificacao[j] == df1.classif[j]:
        total += 1
        
    j+=1 
total

porc_acerto = total/len(df1)
porc_acerto

0.5338983050847458

___
### Concluindo

Neste projeto buscamos fazer uma análise de sentimento, para saber o quanto os usuários estão ajudando a promover o aplicativo spotify. Para obter uma resposta, coletamos tweets que mencionavam o nome do aplicativo para conseguir alcançar um resultado mais especifico.

Começamos nosso trabalho separando os tweets em "relevantes", que mostram o quanto os usuários usam o aplicativo e compartilham com outras pessoas, e os "irrelevantes", que não abordam nenhum conteúdo benéfico para o nome do software. 

A base de Treinamento foi a qual usamos para realizar os cálculos, uma vez que já estava classificada. Devido isso, não poderíamos usar o próprio classificador para gerar mais amostras de treinamento, dado que afetaria nossos resultados pois era de lá que tirávamos nossos números para habilitar o programa. 

Após a classificação dos tweets, fizemos alguns ajustes para a melhoria de nossos resultados. Transformamos toda nossa base de Treinamento em texto corrido, em seguida removemos as pontuações e separamos as palavras para poder calcular a probabilidade do surgimento de cada uma delas, para futuramente usarmos na fórmula naive bayes. Mantivemos os emojis, já que podem ajudar a demonstrar o que o usuário estava sentindo quando escreveu o tweet.

Com essas informações o programa foi capaz de classificar sozinho nossa base de teste. Nosso classificador teve 53% de aperfeiçoamento.

Naive Bayes pode ser usado em diversas situações, como por exemplo na contrução de uma aplicação que detecta SPAM em uma caixa de mensagens. Ou em laboratorios que utilizam essa clafficação para analisar possiveis dados de exames

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html